In [3]:
import numpy as np
import pandas as pd
import csv
import os
import math
from scipy.optimize import fsolve

In [1]:
#finding the coordinates of the B point in the trapezoid
#note 1: p is a dummy variable
#note 2: mn refers to the negative slope
def trapper(p,r1,r2,mn,th2):
    
    cx,cy = r2*np.sin(th2),r2*np.cos(th2)
    bx, by = p
    return (bx**2 + by**2 - r1**2, ((by-cy)/(bx-cx)) - mn )

In [4]:
#Return a dictionary with the total mass and surface area of each configuration
    #Also includes other parameters; specific to the shape

def area(mass=20,        #Target material mass (mass of tantalum, g)
         shape='d',      #Target shape (see options)
         thickness=25,   #Foil thickness (µm)
         m=1,            #Slope for symm cutout OR slope for horseshoe 
         height=0.525,   #Height for D-shaped foils
         r1=0.9144,      #Exterior radius
         r2=0.3644,      #Innermost radius
         length=3.4,     #target chamber length
         th=np.pi/4,     #angle of the symm/horseshoe cutout
         gap=None,       #setting gap between foils if desired
         sep=0           #Separation distance (cm)
        ):
    density   = 16.69 #Tantalum, g/cm3
    volume    = mass/density
    thick_cm  = thickness*1e-4
    
    #Options for shape
    #D shaped
    d_list = ['D','d','D-shaped','d-shaped']
    #Pizza ('symmetrical')
    s_list = ['symm','Symm']
    #Doughnut (with or without horseshoes near the ionizer entrance)
    h_list = ['Donut','donut','doughnut','Doughnut','ring','Ring']
    #Concentric cylinders (This option doesn't work yet – was low priority)
    c_list = ['coil','Coil','cylinder','Cylinder','tube','Tube']
    
    if shape in d_list:
        #area of the piece above the D-shaped foil
        ##note: 'height' is d in source equation (see https://en.wikipedia.org/wiki/Circular_segment)
        a  = r1**2 * (np.arccos(1 - (r1-height)/r1)) - (height) * np.sqrt(r1**2 - height**2) 
        #area of the foil = area of a circle - the area above 
        AREA = np.pi*r1**2 - a
        
        number = volume / (thick_cm*AREA) #find number of foils
        number = math.floor(number)       #integer (floor since we can't use more mass, but can use less)

        tot_area = number * AREA * 2      #total surface area (not including the thin edges)
        
        
        mass = AREA*number*thick_cm*density #actual mass used
        hs,b = 0,0 #unused parameters (for other shapes)
    
    elif shape == 'arc':
        #currently tailored to one arc equation since time was short when I was doing this
        
        #see ./Sketches/Arc-Drawing.jpeg for visual of below equations
        
        #equation of the smaller circle making the arc
        ###x^{2}-1.8x+y^{2}-0.6y=-0.65
        r2 = np.sqrt(0.25)
        c = np.sqrt((0.8923-0.5939)**2 + (0.6953+0.1999)**2)
    
        #segment of large circle that gets cut off
        h1 = r1 - np.sqrt(r1**2 - 0.25*c**2)
        a1 = r1**2 * np.arccos(1-h1/r1) - (r1-h1)*np.sqrt(r1**2 - (r1-h1)**2)
   
        #segment of small circle that gets cut off
        h2 = r2 - np.sqrt(r2**2 - 0.25*c**2)
        a2 = r2**2 * np.arccos(1-h2/r2) - (r2-h2)*np.sqrt(r2**2 - (r2-h2)**2)
        
        #area of circle - segment 1 - segment 2
        AREA = np.pi*r1**2 - a1 - a2
        
        number = volume/(thick_cm*AREA)
        number = math.floor(number)
        
        tot_area = number * AREA * 2
        hs,b = 0,0 #unused parameters
        
    elif shape in s_list:
        #this shape (pizza, symmetrical) is defined by three parameters:
            #1. th (theta, th1 in Inputs code) 
                #Defines 1/2 of the angle of the arc that is cut out of the circle
            #2. m  
                #Defines the slope of the lines in the pizza cutout
                #One line will have slope m, the other will have slope -m
            #3. b (not given by the user in this code or the Inputs code, calculated using th & m.)
                #The x intercept of the two lines in the cutout (x not y; "up" is +x for us)
                #Same for both lines since they intersect at y=0 (x axis)
        #See ./Sketches/Pizza-Drawing.jpeg for a visual of these parameters
        b = r1*np.cos(th) - r1*np.sin(th)*abs(m)
    
        A = (0,b)
        B = (r1*np.sin(th),r1*np.cos(th))
        C = (0,r1)

        #See ./Sketches/Pizza-Drawing.jpeg for a visual of these shapes within the main circle
        #area of the triangle
        tri  = 0.5*abs(A[0]*(B[1]-C[1]) +
                       B[0]*(C[1]-A[1]) + 
                       C[0]*(A[1]-B[1])
                      )
        #segment above chord
        segm = 0.5*r1**2 * (th-np.sin(th))

        #entire circle
        circ = np.pi*r1**2

        AREA = circ - (tri + segm)
        
        number = volume / (thick_cm*AREA) #find number of foils
        number = math.floor(number)       #integer (floor since we can't use more mass, but can use less)

        tot_area = number * AREA * 2      #total surface area (not including the thin edges)
        
        mass = AREA*number*thick_cm*density

        hs = 0 #unused parameter
        
    elif shape in h_list:
        #The doughnut/horseshoe shape is defined by a few parameters
            #r1 (outer radius)
            #r2 (inner radius)
            #m  (slope of line starting from )
        #See ./Sketches/Horseshoe-Drawing.jpeg for a visual
        c1_area = np.pi*r1**2         #large circle area
        c2_area = np.pi*r2**2         #small circle area
        dn_area = c1_area - c2_area   #doughnut area
        
        #finding the area of the missing gap in the horseshoe shape 
        #area = trapezoid + top curve - bottom curve
        
        #trapezoid
        if m > 0: 
            mn = -m
        else:
            mn = m
        #function (defined above) that gets coordinates of B in ./Sketches/Horseshoe-Drawing.jpeg
            #we can use bx,by to find the A-B base needed to calculate the trapezoid's area 
                #note: trapezoid's bases are both horizontal and parallel
        bx, by = fsolve(trapper, (-0.3, 0.6),args=(r1,r2,mn,th))
        
        base1 = 2* r2*np.sin(th)     #C-D base; twice the cx value (th: of inner circle)
        base2 = 2* bx                #A-B base
        
        cy          = r2*np.cos(np.pi/4)
        trap_height = by - cy
        
        trap_area   = 0.5*(base1+base2)*trap_height
        
        #curves: again see https://en.wikipedia.org/wiki/Circular_segment for equation and h value meaning
            #h1 = r1 - by
        curve1_area = r1**2 * (np.arccos(1 - (r1-by)/r1)) - (by) * np.sqrt(r1**2 - by**2)
            #h2 = r2 - cy
        curve2_area = r2**2 * (np.arccos(1 - (r2-cy)/r2)) - (cy) * np.sqrt(r2**2 - cy**2)
        
        weird_area  = trap_area + curve1_area - curve2_area
        
        #horseshoe area
        hs_area = dn_area - weird_area #doughnut area - cutout area
        
        #setting up horseshoe parameters
        hs = {
            'HS_num':None,     #number of horseshoe foils
            'DN_num':None,     #number of doughnut foils
            'HS_area':hs_area, #area of one side of horseshoe foils
            'DN_area':dn_area  #area of one side of doughnut foils
        }
        
        #hs fraction of foils: sep/length
        #dn fraction of foils: (length-sep)/length
        AREA   = (sep/length)*hs_area + ((length-sep)/length)*dn_area  
        #average area per foil side (to calculate total area; relevant single surf. areas are in hs dict)
        number = volume / (thick_cm*AREA) #find number of foils
        number = math.floor(number)       #integer (floor since we can't use more mass, but can use less)

        tot_area = number * AREA * 2      #total surface area (not including the thin edges)
        
        hs['HS_num'] = math.floor(sep/length * number)
        hs['DN_num'] = number - hs['HS_num']
        
        mass = AREA*number*thick_cm*density

        b=None #unused parameter
    elif shape == 'cern':
        #to validate the number calculations  
        #also to get total surface area of the CERN setup
        AREA = 1*15
        
        number = volume / (thick_cm*AREA) #find number of foils
        number = math.floor(number)       #integer (floor since we can't use more mass, but can use less)

        tot_area = number * AREA * 2      #total surface area (not including the thin edges)
        
        mass = AREA*number*thick_cm*density
        
        hs,b = 0,0 #unused parameters
        
    else: #shape is in c_list
        #ABANDONED FOR NOW: was low priority. 
        #concentric cylinders (approximating a spiral design)
        area   = 2*pi
        
        #THIS IS GONNA HAVE TO BE A LOOP OF SOME SORT WITH THE RADII 
        #big question: how to find the number based on area when area is highly dependent on the number of foils
        
        number = volume / (thick_cm*AREA) #find number of foils
        number = math.floor(number)       #integer (floor since we can't use more mass, but can use less)

        tot_area = number * AREA * 2      #total surface area (not including the thin edges)
 
        mass = AREA*number*thick_cm*density
    
        hs,b = 0,0 #unused parameters
    
    #if gap is given, calculate new length value
    if gap != None:
        length = gap* (number+1) + number*thick_cm + sep 
    #if gap is not given, calculate it based on given length
    else:
        gap = (length-(number*thickness*1e-4)-sep)/(number+1)
    
    #create dictionary of values to report
    things = {
        'Area':AREA,
        'Number':number,
        'Total Area':tot_area,
        'Gap':gap,
        'Length':length,
        'Horseshoes':hs,
        'Mass':mass,
        'b':b
    }
    return things

### Thick (Run 4)

In [213]:
l = 3.4
g = 0.004723991507430997
t = 2e-4

n = math.floor((l-g)/(g+t))
a1 = 2.217779639723998
m = 16.69*n*a1*t
sa = a1*2*n
print(f'Mass: {m:.4}')
print(f'Area/foil side: {a1:.4}')
print(f'Tot Surf Area: {sa:.4}')
print(f'Number: {n}')

Mass: 5.101
Area/foil side: 2.218
Tot Surf Area: 3.056e+03
Number: 689


### Thick (Run 5)

In [214]:
area(mass=43.49,thickness=2,gap=0.004723991507430997)

{'Area': 2.217779639723998,
 'Number': 5874,
 'Total Area': 26054.475207477528,
 'Gap': 0.004723991507430997,
 'Length': 28.92825010615711,
 'Horseshoes': 0,
 'Mass': 43.484919121279994,
 'b': 0}

### Thickness (Run 6)

In [217]:
a1 = 2.217779639723998
saC = 5970
t = 2e-4
g = 0.004723991507430997
num = math.floor(saC/(2*a1))
length = num*(g+t)-g
mm = 16.69*num*a1*t
area(mass=mm,length=length,thickness=2)

{'Area': 2.217779639723998,
 'Number': 1345,
 'Total Area': 5965.827230857554,
 'Gap': 0.004716972203556656,
 'Length': 6.618044585987259,
 'Horseshoes': 0,
 'Mass': 9.956965648301258,
 'b': 0}

## Arc (Run 7)

In [240]:
area(mass=43.49,shape='arc')

c= 0.9436236537942444
a1= 0.08374846092814647
h2= 0.3344898794635205
0.5 0.16551012053647954 0.02739359999999999
0.23026330803471856 1.2334117614999915
2.3127598026785634


{'Area': 2.3127598026785634,
 'Number': 450,
 'Total Area': 2081.483822410707,
 'Gap': 0.005044345898004434,
 'Length': 3.4,
 'Horseshoes': 0,
 'Mass': 43.49,
 'b': 0}

## Thin (Run w/ Aurelia)

In [242]:
area(mass=43.49,shape='D',thickness=2,length=30)

{'Area': 2.217779639723998,
 'Number': 5874,
 'Total Area': 26054.475207477528,
 'Gap': 0.004906417021276596,
 'Length': 30,
 'Horseshoes': 0,
 'Mass': 43.484919121279994,
 'b': 0}